# Tensorflow 2.0 tutorial

## Beginner tutorial codes
from [tensorflow coalb](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/quickstart/beginner.ipynb#scrollTo=7NAbSZiaoJ4z)

In [2]:
import tensorflow as tf

In [3]:
mnist = tf.keras.datasets.mnist # data load

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 # 0-1 scaling

In [5]:
x_train.shape # data check

(60000, 28, 28)

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
]) # build model with keras wrapper

In [7]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy']) # need to refer keras cook book

In [8]:
model.fit(x_train, y_train, epochs=5) # train model

W0620 21:52:05.030114 139958398957376 deprecation.py:323] From /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 41us/sample - loss: 0.2942 - accuracy: 0.9141
Epoch 2/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.1441 - accuracy: 0.9575
Epoch 3/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.1083 - accuracy: 0.9676
Epoch 4/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.0890 - accuracy: 0.9727
Epoch 5/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.0763 - accuracy: 0.9759


In [9]:
model.evaluate(x_test, y_test) 

10000/10000 [==============================] - 0s 23us/sample - loss: 0.0763 - accuracy: 0.9766


[0.07630281789442524, 0.9766]

loss value ~ 0.07, ~98% accuracy with simple FC ANN model

## now expert mode
from [this colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/quickstart/advanced.ipynb)

In [10]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [11]:
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [12]:
x_train.shape # data dimenstion check

(60000, 28, 28, 1)

In [13]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [14]:
train_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>

In [15]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
# build modes with class
model = MyModel()

In [16]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [17]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
# for evaluation metrix
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [21]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions) 
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss) # for evaluation metric
    train_accuracy(labels, predictions) # # for evaluation metric

In [19]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)
# note that there's no gradients or optimizer function, 
# just predictions and loss calculations.
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [22]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

Epoch 1, Loss: 0.1368824541568756, Accuracy: 95.93499755859375, Test Loss: 0.06352698057889938, Test Accuracy: 97.83999633789062
Epoch 2, Loss: 0.08870803564786911, Accuracy: 97.3550033569336, Test Loss: 0.062405869364738464, Test Accuracy: 97.95500183105469
Epoch 3, Loss: 0.06583841890096664, Accuracy: 98.04000091552734, Test Loss: 0.06571976095438004, Test Accuracy: 97.97999572753906
Epoch 4, Loss: 0.052594494074583054, Accuracy: 98.41541290283203, Test Loss: 0.06651460379362106, Test Accuracy: 98.02999877929688
Epoch 5, Loss: 0.043957341462373734, Accuracy: 98.66899871826172, Test Loss: 0.06656648963689804, Test Accuracy: 98.0719985961914


* slightly better performance than that of FC ANN model.